# Volume scan from  multiple sweeps using xradar
This example shows how to create a volume scan from multiple sweep files stored on AWS. The volume scan structure is based on [tree-like](https://xarray-datatree.readthedocs.io/en/latest/generated/datatree.DataTree.html) hierarchical collection of xarray objects 

## Imports

In [1]:
import fsspec
import xradar as xd
import xarray as xr
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import warnings
from pandas import to_datetime
from datetime import datetime
from matplotlib import pyplot
from datatree import DataTree, open_datatree
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
warnings.simplefilter("ignore")
# pyplot.style.use('dark_background')

## Access radar data from the Colombian radar network on AWS
Access data from the IDEAM bucket on AWS. Detailed information can be found [here](https://openradar-docs--102.org.readthedocs.build/projects/xradar/en/102/notebooks/Read-plot-Sigmet-data-from-AWS.html) 

In [2]:
def create_query(date, radar_site):
    """
    Creates a string for quering the IDEAM radar files stored in AWS bucket
    :param date: date to be queried. e.g datetime(2021, 10, 3, 12). Datetime python object
    :param radar_site: radar site e.g. Guaviare
    :return: string with a IDEAM radar bucket format
    """
    if (date.hour != 0) and (date.hour != 0):
        return f'l2_data/{date:%Y}/{date:%m}/{date:%d}/{radar_site}/{radar_site[:3].upper()}{date:%y%m%d%H}'
    elif (date.hour != 0) and (date.hour == 0):
        return f'l2_data/{date:%Y}/{date:%m}/{date:%d}/{radar_site}/{radar_site[:3].upper()}{date:%y%m%d}'
    else:
        return f'l2_data/{date:%Y}/{date:%m}/{date:%d}/{radar_site}/{radar_site[:3].upper()}{date:%y%m%d}'

In [3]:
date_query = datetime(2023, 4, 7, 3)
radar_name = "Barrancabermeja" 
query = create_query(date=date_query, radar_site=radar_name)
str_bucket = 's3://s3-radaresideam/'
fs = fsspec.filesystem("s3", anon=True)

In [4]:
radar_files = sorted(fs.glob(f"{str_bucket}{query}*"))

In [5]:
def data_accessor(file):
    """
    Open AWS S3 file(s), which can be resolved locally by file caching
    """
    return fsspec.open_local(f'simplecache::s3://{file}', s3={'anon': True}, filecache={'cache_storage': '/tmp/radar/'})


def create_vcp(ls_dt):
    """
    Creates a tree-like object for each volume scan
    """
    return DataTree(name='root', children=dict(SURVP=ls_dt[0], PRECA=ls_dt[1], PRECB=ls_dt[2], PRECC=ls_dt[3]))


def create_path(dt):
    dt = pd.to_datetime(dt.sweep_0.time.values[0])
    return f"{dt:%Y%m%d%H%M}", dt


def mult_vcp(radar_files):
    """
    Creates a tree-like object for multiple volumes scan every 4th file in the bucket
    """
    paths = []
    ls_time = []
    dt_dict = {}
    for idx, i in enumerate(radar_files):
        if idx % 4 == 0:
            ls_files = radar_files[idx: idx + 4]
            ls_sigmet = [xd.io.open_iris_datatree(data_accessor(j)).xradar.georeference() for j in ls_files]
            vcp = create_vcp(ls_sigmet)
            path, time = create_path(vcp['SURVP'])
            paths.append(path)
            ls_time.append(time)
            dt_dict[f"{path}"] = vcp

    root_ds = xr.Dataset(
        {
            "vcp_time": xr.DataArray(
                paths,
                dims=["time"],
                coords={"time": pd.Series(list(ls_time)).sort_values()},
            ),
        },
        coords={
            "time": pd.Series(list(ls_time)).sort_values(),
        },
    )
    root_ds = root_ds.sortby("time")
    dt_dict["/"] = root_ds
    return DataTree.from_dict(dt_dict)

In [6]:
# let's test it using the first 24 files in the bucket. We can include more files for visualization. e.g. radar_files[:96]
vcps_dt = mult_vcp(radar_files[:96])
list(vcps_dt.groups)

['/',
 '/202304070300',
 '/202304070300/SURVP',
 '/202304070300/SURVP/sweep_0',
 '/202304070300/PRECA',
 '/202304070300/PRECA/sweep_0',
 '/202304070300/PRECA/sweep_1',
 '/202304070300/PRECA/sweep_2',
 '/202304070300/PRECA/sweep_3',
 '/202304070300/PRECB',
 '/202304070300/PRECB/sweep_0',
 '/202304070300/PRECB/sweep_1',
 '/202304070300/PRECC',
 '/202304070300/PRECC/sweep_0',
 '/202304070300/PRECC/sweep_1',
 '/202304070300/PRECC/sweep_2',
 '/202304070305',
 '/202304070305/SURVP',
 '/202304070305/SURVP/sweep_0',
 '/202304070305/PRECA',
 '/202304070305/PRECA/sweep_0',
 '/202304070305/PRECA/sweep_1',
 '/202304070305/PRECA/sweep_2',
 '/202304070305/PRECA/sweep_3',
 '/202304070305/PRECB',
 '/202304070305/PRECB/sweep_0',
 '/202304070305/PRECB/sweep_1',
 '/202304070305/PRECC',
 '/202304070305/PRECC/sweep_0',
 '/202304070305/PRECC/sweep_1',
 '/202304070305/PRECC/sweep_2',
 '/202304070310',
 '/202304070310/SURVP',
 '/202304070310/SURVP/sweep_0',
 '/202304070310/PRECA',
 '/202304070310/PRECA/sweep_

In [7]:
times = list(vcps_dt.children)
times 

['202304070300',
 '202304070305',
 '202304070310',
 '202304070316',
 '202304070320',
 '202304070325',
 '202304070330',
 '202304070335',
 '202304070340',
 '202304070345',
 '202304070350',
 '202304070356']

## Multiple volumes scan into one `datatree` object

Similarly, we can create a tree-like hierarchical object with multiple volume scans.

Now we have 6 vcps in one tree-like hierarchical object.

In [8]:
print(f"Size of data in tree = {vcps_dt.nbytes / 1e9 :.2f} GB")

Size of data in tree = 2.31 GB


### PPI animation using the lowest elevation angle

We can create an animation using the `FuncAnimation` module from `matplotlib` package

In [9]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
proj_crs = xd.georeference.get_crs(vcps_dt[times[0]].SURVP)
cart_crs = ccrs.Projection(proj_crs)
sc = vcps_dt[times[0]].SURVP.sweep_0.DBZH.plot.pcolormesh(x="x", y="y", vmin=-10, 
                                                      vmax=50, cmap="Spectral_r", 
                                                      edgecolors="face", 
                                                      transform=cart_crs,
                                                      ax=ax)

title = f"SURVP - {vcps_dt[times[0]].SURVP.sweep_0.sweep_fixed_angle.values: .1f} [deg] \n {pd.to_datetime(vcps_dt[times[0]].SURVP.sweep_0.time.values[0]):%Y-%m-%d %H:%M}UTC"
ax.set_title(title)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, 
                  linewidth=1, color="gray", alpha=0.3, linestyle="--")
plt.gca().xaxis.set_major_locator(plt.NullLocator())
gl.top_labels = False
gl.right_labels = False
ax.coastlines()

def update_plot(t):
    sc.set_array(vcps_dt[t].SURVP.sweep_0.DBZH.values.ravel())
    ax.set_title(f"SURVP - {vcps_dt[t].SURVP.sweep_0.sweep_fixed_angle.values: .1f} [deg] \n {pd.to_datetime(vcps_dt[t].SURVP.sweep_0.time.values[0]):%Y-%m-%d %H:%M}UTC")

ani = FuncAnimation(fig, update_plot, frames=times,
                    interval=150)
plt.close()
display(HTML(ani.to_html5_video()))

## Bonus: Analysis-ready data, cloud-optimized (ARCO) format 

Tree-like hierarchical data can be stored using ARCO format.

In [10]:
zarr_store = './multiple_vcp_test.zarr'
# _ = vcps_dt.to_zarr(zarr_store)

In [11]:
! ls multiple_vcp_test.zarr

202304070300  202304070310  202304070320  time
202304070305  202304070316  202304070325  vcp_time


Read ARCO format data using `open_datatree`

In [ ]:
vcps_back = open_datatree(zarr_store, engine="zarr")

In [ ]:
display(vcps_back)